# Lab 6

Advanced APIs for Scale - Batching

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv(override=True)

openai = OpenAI()


In [2]:
questions = [
    "What is the capital of France?",
    "What is the capital of Germany?",
    "What is the capital of Italy?",
    "What is the capital of Spain?",
    "What is the capital of Portugal?",
    "What is the capital of Greece?",
    "What is the capital of Turkey?",
    "What is the capital of Egypt?",
    "What is the capital of South Africa?",
]

## First step - Generate a jsonl file

With this format:

```jsonl
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
```

In [ ]:
def make_line(index):
    question = questions[index]
    return f'{{"custom_id": "request-{index}", "method": "POST", "url": "/v1/chat/completions", "body": {{"model": "gpt-4.1-nano", "messages": [{{"role": "user", "content": "{question}"}}],"max_tokens": 1000}}}}'


make_line(0)

In [4]:
with open("questions.jsonl", "w", encoding="utf-8") as f:
    for i in range(len(questions)):
        f.write(make_line(i) + "\n")

## Second step - upload the file

In [ ]:

questions_file = openai.files.create(file=open("questions.jsonl", "rb"), purpose="batch")
questions_file

## Third step - make the batch

In [ ]:


file_id = questions_file.id
batch = openai.batches.create(input_file_id=file_id, endpoint="/v1/chat/completions", completion_window="24h")
batch

## Fourth step - monitor the batch

In [ ]:
latest = openai.batches.retrieve(batch.id)
print(latest)
print(latest.status)

## Fifth step - collect the results

In [ ]:
results_file_id = latest.output_file_id
results_file = openai.files.content(results_file_id)
results_file.text

## And now let's see

In [ ]:
import json
lines = results_file.text.split("\n")
results = [json.loads(line) for line in lines if line]
print(f"Number of results: {len(results)}")
print("First result:")
results[0]

In [ ]:
for result in results:
    id = result["custom_id"]
    question = questions[int(id.split("-")[1])]
    print(question)
    print(result["response"]["body"]["choices"][0]["message"]["content"])
    print("-"*100)

## Anthropic has a similar Batch API

https://docs.anthropic.com/en/docs/build-with-claude/batch-processing